# Chapter 1 - Developing LMM Application with LangChain

Welcome to the LangChain framework for building applications on LLMs! You'll learn about the main components of LangChain, including models, chains, agents, prompts, and parsers. You'll create chatbots using both open-source models from Hugging Face and proprietary models from OpenAI, create prompt templates, and integrate different chatbot memory strategies to manage context and resources during conversations.

In [ ]:
import sys
import os
from openai import OpenAI

# Use current working directory and go one level up
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

# Now you can import your config
from config import api_key

client = OpenAI(api_key=api_key)

### Section 1.1 - The langChain ecosystem

#### OpenAI models in LangChain!

OpenAI's GPT-series models are some of the highest performing LLMs around. They are available via OpenAI's API, which you can easily interact with using LangChain.

Normally, using OpenAI's models would require a personal API key used for billing the cost of the models. In this course, you do not need to create or provide an OpenAI API key. The `"<OPENAI_API_TOKEN>"` placeholder will send valid requests to the API. If you make large number of requests in a short period, you may encounter a `RateLimitError`. If you see this, please pause for a moment and try again.

The `ChatOpenAI` class has already been imported.

In [ ]:
from langchain_openai import ChatOpenAI

# Define the LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

# Predict the words following the text in question
prompt = 'Three reasons for using LangChain for LLM application development.'
response = llm.invoke(prompt)

print(response.content)

#### Hugging Face models in LangChain!

There are thousands of models freely available to download and use on Hugging Face. Hugging Face integrates really nicely into LangChain via its partner library, `langchain-huggingface`, which is available for you to use.

In this exercise, you'll load and call the `crumb/nano-mistral` model from Hugging Face. This is a ultra-light LLM designed to be fine-tuned for greater performance.

In [ ]:
# Import the class for defining Hugging Face pipelines
from langchain_huggingface import HuggingFacePipeline

# Define the LLM from the Hugging Face model ID
llm = HuggingFacePipeline.from_model_id(
    model_id="crumb/nano-mistral",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 20}
)

prompt = "Hugging Face is"

# Invoke the model
response = llm.invoke(prompt)
print(response)

### Section 1.2 - Prompt templates

In [ ]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

#### Prompt templates and chaining

In this exercise, you'll begin using two of the core components in LangChain: prompt templates and chains!

The classes necessary for completing this exercise, including `ChatOpenAI`, have been pre-loaded for you.

In [ ]:
from langchain_core.prompts import PromptTemplate

# Create a prompt template from the template string
template = "You are an artificial intelligence assistant, answer the question. {question}"
prompt = PromptTemplate.from_template(
    template=template
)

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)	

# Create a chain to integrate the prompt template and LLM
llm_chain = prompt | llm

# Invoke the chain on the question
question = "How does LangChain make LLM application development easier?"
response = llm_chain.invoke({"question": question})

In [ ]:
print(response.content)

#### Chat prompt templates

Given the importance of chat models in many LLM applications, LangChain provides functionality for creating prompt templates to structure messages to different chat roles.

The `ChatPromptTemplate` class has already been imported for you, and an LLM has already been defined.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

# Create a chat prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert that returns the colors present in a country's flag."),
        ("human", "France"),
        ("ai", "blue, white, red"),
        ("human", "{country}")
    ]
)

prompt = prompt_template.invoke({"country": "France"})

In [ ]:
print(prompt.model_dump()['messages'])

In [ ]:
# Chain the prompt template and model, and invoke the chain
llm_chain = prompt_template | llm

country = "Japan"
response = llm_chain.invoke({"country": country})
print(response.content)

### Section 1.3 - Few-shot Prompting

#### Creating the few-shot example set
PromptTemplate and ChatPromptTemplate are great for integrating variables, but struggle with integrating datasets containing many examples. This is where `FewShotPromptTemplate` comes in! In this exercise, you'll create a dataset, in the form of a list of dictionaries, to contain the follow question-answer pairs.

- Question: How many DataCamp courses has Jack completed?
- Answer: 36
- Question: How much XP does Jack have on DataCamp?
- Answer: 284,320XP
- Question: What technology does Jack learn about most on DataCamp?
- Answer: Python
- 
In the next exercise, you'll convert this information into a few-shot prompt template.

In [ ]:
# Create the examples list of dicts
examples = [
  {
    "question": "How many DataCamp courses has Jack completed?",
    "answer": "36"
  },
  {
    "question": "How much XP does Jack have on DataCamp?",
    "answer": "284,320XP"
  },
  {
    "question": "What technology does Jack learn about most on DataCamp?",
    "answer": "Python"
  }
]

#### Building the few-shot prompt template

With your examples in a structured format, it's now time to create the few-shot prompt template! You'll create a template that convert the question-answer pairs into the following format:

```
Question: Example question
Example Answer

``` 
All of the LangChain classes necessary for completing this exercise have been pre-loaded for you.

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate

# Complete the prompt for formatting answers
example_prompt = PromptTemplate.from_template("Question: {question}\n{answer}")

# Create the few-shot prompt
prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

prompt = prompt_template.invoke({"input": "What is Jack's favorite technology on DataCamp?"})
print(prompt.text)

#### Implementing few-shot prompting
Time to combine your components together into a chain! The few-shot prompt you created in the previous exercise is still available for you to use, along with `examples` and `example_prompt`.

All of the LangChain classes necessary for completing this exercise have been pre-loaded for you.

In [ ]:
prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

# Create an OpenAI chat LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

# Create and invoke the chain
llm_chain = prompt_template | llm
response = llm_chain.invoke({"input": "What is Jack's favorite technology on DataCamp?"})

In [ ]:
response.content

## Chapter 2 - Chains and Agents

Time to level up your LangChain chains! You'll learn to use the LangChain Expression Language (LCEL) for defining chains with greater flexibility. You'll create sequential chains, where inputs are passed between components to create more advanced applications. You'll also begin to integrate agents, which use LLMs for decision-making.

### Section 2.1 - Sequential chains

#### Building prompts for sequential chains

Over the next couple of exercises, you'll work to create a system for helping people learn new skills. This system needs to be built sequentially, so learners can modify plans based on their preferences and constraints. You'll utilize your LangChain LCEL skills to build a sequential chain to build this system, and the first step is to design the prompt templates that will be used by this system.

In [ ]:
# Create a prompt template that takes an input activity
learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

# Create a prompt template that places a time constraint on the output
time_prompt = PromptTemplate(
    input_variables=['learning_plan'],
    template="I only have one week. Can you create a plan to help me hit this goal: {learning_plan}."
)

# Invoke the learning_prompt with an activity
print(learning_prompt.invoke({"activity": "play golf"}))

#### Sequential chains with LCEL
With your prompt templates created, it's time to tie everything together, including the LLM, using chains and LCEL. An `llm` has already been defined for you that uses OpenAI's gpt-4o-mini model

For the final step of calling the chain, feel free to insert any activity you wish! If you're struggling for ideas, try inputting `"play the harmonica"`.

In [ ]:
from langchain.schema.output_parser import StrOutputParser

# Create an OpenAI chat LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)


In [ ]:
learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

time_prompt = PromptTemplate(
    input_variables=["learning_plan"],
    template="I only have one week. Can you create a concise plan to help me hit this goal: {learning_plan}."
)

# Complete the sequential chain with LCEL
seq_chain = ({"learning_plan": learning_prompt | llm | StrOutputParser()}
    | time_prompt
    | llm
    | StrOutputParser())

# Call the chain
print(seq_chain.invoke({"activity": "lay the harmonica"}))

so basically the first part of the pipeline is get a response on learning the activity 

In [ ]:
(learning_prompt | llm | StrOutputParser()).invoke("play golf")

### Section 2.2 - Introduction to LangChain agents

#### ReAct agents

Time to have a go at creating your own ReAct agent! Recall that ReAct stands for Reason and Act, which describes how they make decisions. In this exercise, you'll load the built-in `wikipedia` tool to integrate external data from Wikipedia with your LLM. An `llm` has already been defined for you that uses OpenAI's gpt-4o-mini model

Note: The `wikipedia` tool requires the `wikipedia` Python library to be installed as a dependency, which has been done for you in this case.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.load_tools import load_tools
from langgraph.prebuilt import create_react_agent

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

In [ ]:
# Define the tools
tools = load_tools(["wikipedia"])

# Define the agent
agent = create_react_agent(llm, tools)

# Invoke the agent
response = agent.invoke({"messages": [("human", "How many people live in Amsterdam?")]})
print(response["messages"][-1].content)

### Section 2.2 - Custom tools for agents


#### Defining a function for tool use
You're working for a SaaS (software as a service) company with big goals for rolling out tools to help employees at all levels of the organization to make data-informed decisions. You're creating a PoC for an application that allows customer success managers to interface with company data using natural language to retrieve important customer data.

You've been provided with a pandas DataFrame called customers that contains a small sample of `customer` data. Your first step in this project is to define a Python function to extract information from this table given a customer's name. `pandas` has already been imported as `pd`.

In [ ]:
import pandas as pd

In [ ]:
customers = pd.read_csv('customers.csv')

In [ ]:
# Define a function to retrieve customer info by-name
def retrieve_customer_info(name: str) -> str:
    """Retrieve customer information based on their name."""
    # Filter customers for the customer's name
    customer_info = customers[customers['name'] == name]
    return customer_info.to_string()
  
# Call the function on Peak Performance Co.
print(retrieve_customer_info("Peak Performance Co."))

#### Creating custom tools

Now that you have a function for extracting customer data from the `customers` DataFrame, it's time to convert this function into a tool that's compatible with LangChain agents.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_community.agent_toolkits.load_tools import load_tools
from langgraph.prebuilt import create_react_agent

In [ ]:
# Convert the retrieve_customer_info function into a tool
@tool
def retrieve_customer_info(name: str) -> str:
    """Retrieve customer information based on their name."""
    customer_info = customers[customers['name'] == name]
    return customer_info.to_string()
  
# Print the tool's arguments
print(retrieve_customer_info.args)

#### Integrating custom tools with agents

Now that you have your tools at-hand, it's time to set up your agentic workflow! You'll again be using a ReAct agent, which, recall, reasons on the steps it should take, and selects tools using this context and the tool descriptions. An `llm` has already been defined for you that uses OpenAI's `gpt-4o-mini` model.

In [ ]:
@tool
def retrieve_customer_info(name: str) -> str:
    """Retrieve customer information based on their name."""
    customer_info = customers[customers['name'] == name]
    return customer_info.to_string()

# Create a ReAct agent
agent = create_react_agent(llm, tools=[retrieve_customer_info])

# Invoke the agent on the input
messages = agent.invoke({"messages": [("human", "Create a summary of our customer: Peak Performance Co.")]})
print(messages['messages'][-1].content)

## Chapter 3 - Retreival Augmented Generation (RAG)

One limitation of LLMs is that they have a knowledge cut-off due to being trained on data up to a certain point. In this chapter, you'll learn to create applications that use Retrieval Augmented Generation (RAG) to integrate external data with LLMs. The RAG workflow contains a few different processes, including splitting data, creating and storing the embeddings using a vector database, and retrieving the most relevant information for use in the application. You'll learn to master the entire workflow!

### Section 3.1. - Integrating document loaders

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, CSVLoader, UnstructuredHTMLLoader
loader = CSVLoader('customers.csv')
data = loader.load()
data

#### PDF document loaders

To begin implementing Retrieval Augmented Generation (RAG), you'll first need to load the documents that the model will access. These documents can come from a variety of sources, and LangChain supports document loaders for many of them.

In this exercise, you'll use a document loader to load a PDF document containing the paper, RAG VS Fine-Tuning: Pipelines, Tradeoffs, and a Case Study on Agriculture by Balaguer et al. (2024).

Note: `pypdf`, a dependency for loading PDF documents in LangChain, has already been installed for you.

In [ ]:
# Import library
from langchain_community.document_loaders import PyPDFLoader

# Create a document loader for rag_vs_fine_tuning.pdf
loader = PyPDFLoader("./data/rag_vs_fine_tuning.pdf")

# Load the document
data = loader.load()
print(data[0])

#### CSV document loaders

Comma-separated value (CSV) files are an extremely common file format, particularly in data-related fields. Fortunately, LangChain provides different document loaders for different formats, keeping almost all of the syntax the same!

In this exercise, you'll use a document loader to load a CSV file containing data on FIFA World Cup international viewership. If you're interested in the full analysis behind this data, check out How to Break FIFA.

In [ ]:
# Import library
from langchain_community.document_loaders.csv_loader import CSVLoader

# Create a document loader for fifa_countries_audience.csv
loader = CSVLoader("./data/fifa_countries_audience.csv")

# Load the document
data = loader.load()
print(data[0])

#### HTML document loaders

It's possible to load documents from many different formats, including complex formats like HTML.

In this exercise, you'll load an HTML file containing a White House executive order.

In [ ]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

# Create a document loader for unstructured HTML
loader = UnstructuredHTMLLoader("./data/white_house_executive_order_nov_2023.html")

# Load the document
data = loader.load()

# Print the first document
print(data[0])

# Print the first document's metadata
print(data[0].metadata)

### Section 3.2 - Splitting external data for retrieval

#### Splitting by character
A key process in implementing Retrieval Augmented Generation (RAG) is splitting documents into chunks for storage in a vector database.

There are several splitting strategies available in LangChain, some with more complex routines than others. In this exercise, you'll implement a character text splitter, which splits documents based on characters and measures the chunk length by the number of characters.

Remember that there is no ideal splitting strategy, you may need to experiment with a few to find the right one for your use case.

In [ ]:
# Import the character splitter
from langchain_text_splitters import CharacterTextSplitter

quote = 'Words are flowing out like endless rain into a paper cup,\nthey slither while they pass,\nthey slip away across the universe.'
chunk_size = 24
chunk_overlap = 10

# Create an instance of the splitter class
splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=24,
    chunk_overlap=10
)

# Split the string and print the chunks
docs = splitter.split_text(quote)
print(docs)
print([len(doc) for doc in docs])

#### Recursively splitting by character

Many developers are using a recursive character splitter to split documents based on a specific list of characters. These characters are paragraphs, newlines, spaces, and empty strings, by default: `["\n\n", "\n", " ", ""]`.

Effectively, the splitter tries to split by paragraphs, checks to see if the `chunk_size` and `chunk_overlap` values are met, and if not, splits by sentences, then words, and individual characters.

Often, you'll need to experiment with different `chunk_size` and chunk_overlap values to find the ones that work well for your documents.

In [ ]:
# Import the recursive character splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

quote = 'Words are flowing out like endless rain into a paper cup,\nthey slither while they pass,\nthey slip away across the universe.'
chunk_size = 24
chunk_overlap = 10

# Create an instance of the splitter class
splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

# Split the document and print the chunks
docs = splitter.split_text(quote)
print(docs)
print([len(doc) for doc in docs])

#### Splitting HTML
In this exercise, you'll split an HTML containing an executive order on AI created by the US White House in October 2023. To retain as much context as possible in the chunks, you'll split using larger chunk_size and chunk_overlap values.

All of the LangChain classes necessary for completing this exercise have been pre-loaded for you.

In [ ]:
# Load the HTML document into memory
loader = UnstructuredHTMLLoader("./data/white_house_executive_order_nov_2023.html")
data = loader.load()

# Define variables
chunk_size = 300
chunk_overlap = 100

# Split the HTML
splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=['.'])

docs = splitter.split_documents(data) 
print(docs)

### Section 3.3 - RAG storage and retrieval using vector databases

#### Preparing the documents and vector database
Over the next few exercises, you'll build a full RAG workflow to have a conversation with a PDF document containing the paper, RAG VS Fine-Tuning: Pipelines, Tradeoffs, and a Case Study on Agriculture by Balaguer et al. (2024). This works by splitting the documents into chunks, storing them in a vector database, defining a prompt to connect the retrieved documents and user input, and building a retrieval chain for the LLM to access this external data.

In this exercise, you'll prepare the document for storage and ingest them into a Chroma vector database. You'll use a RecursiveCharacterTextSplitter to chunk the PDF, and ingest them into a Chroma vector database using an OpenAI embeddings function. As with the rest of the course, you don't need to provide your own OpenAI API key.

The following classes have already been imported for you: `RecursiveCharacterTextSplitter`, `Chroma`, and `OpenAIEmbeddings`.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [ ]:
loader = PyPDFLoader('./data/rag_vs_fine_tuning.pdf')
data = loader.load()

# Split the document using RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=300,
    chunk_overlap=50
)
docs = splitter.split_documents(data) 

# Embed the documents in a persistent Chroma vector database
embedding_function = OpenAIEmbeddings(api_key=api_key, model='text-embedding-3-small')
vectorstore = Chroma.from_documents(
    docs,
    embedding=embedding_function,
    persist_directory="./chroma"
)

# Configure the vector store as a retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

#### Building a retrieval prompt template
Now your documents have been ingested into vector database and are ready for retrieval, you'll need to design a chat prompt template to combine the retrieved document chunks with the user input question.

The general structure of the prompt has already been provided; your goal is to insert the correct input variable placeholders into the `message` string and convert the string into a chat prompt template.

In [ ]:
# Add placeholders to the message string
message = """
Answer the following question using the context provided:

Context:
{context}

Question:
{question}

Answer:
"""

# Create a chat prompt template from the message string
prompt_template = ChatPromptTemplate.from_messages([("human", message)])

In [ ]:
print(prompt_template)

#### Creating a RAG chain
Now to bring all the components together in your RAG workflow! You've prepared the documents and ingested them into a Chroma database for retrieval. You created a prompt template to include the retrieved chunks from the academic paper and answer questions.

The prompt template you created in the previous exercise is available as prompt_template, an OpenAI model has been initialized as `llm`, and the code to recreate your `retriever` has be included in the script.

In [ ]:
from langchain_core.runnables import RunnablePassthrough

vectorstore = Chroma.from_documents(
    docs,
    embedding=OpenAIEmbeddings(api_key=api_key, model='text-embedding-3-small'),
    persist_directory="./chroma/"
)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# Create a chain to link retriever, prompt_template, and llm
rag_chain = ({"context": retriever, "question": RunnablePassthrough()}
            | prompt_template
            | llm)

# Invoke the chain
response = rag_chain.invoke("Which popular LLMs were considered in the paper?")
print(response.content)